In [1]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np
import random

In [2]:
import pandas as pd

df = pd.read_csv("the-circor-digiscope-phonocardiogram-dataset-1.0.3/training_data.csv")
df

,Patient ID,Recording locations:,Age,Sex,Height,Weight,Pregnancy status,Murmur,Murmur locations,Most audible location,...,Systolic murmur pitch,Systolic murmur quality,Diastolic murmur timing,Diastolic murmur shape,Diastolic murmur grading,Diastolic murmur pitch,Diastolic murmur quality,Outcome,Campaign,Additional ID
0,2530,AV+PV+TV+MV,Child,Female,98.0,15.9,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
1,9979,AV+PV+TV+MV,Child,Female,103.0,13.1,False,Present,AV+MV+PV+TV,TV,...,High,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
2,9983,AV+PV+TV+MV,Child,Male,115.0,19.1,False,Unknown,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
3,13918,AV+PV+TV+MV,Child,Male,98.0,15.9,False,Present,TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
4,14241,AV+PV+TV+MV,Child,Male,87.0,11.2,False,Present,AV+MV+PV+TV,PV,...,Low,Harsh,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
937,85340,AV+PV+TV+MV,Child,Male,105.0,16.6,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
938,85341,AV+PV+TV+MV,Child,Male,92.0,15.2,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN
939,85343,AV+PV+TV+MV,Child,Female,97.0,13.5,False,Present,MV+TV,TV,...,Low,Blowing,NaN,NaN,NaN,NaN,NaN,Abnormal,CC2015,NaN
940,85345,AV+PV,Child,Female,132.0,38.1,False,Absent,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Normal,CC2015,NaN


In [3]:
patient_id = list(df["Patient ID"])
recording_loc = list(df["Recording locations:"])
murmur = list(df["Murmur"])
murmur_loc = list(df["Murmur locations"])
df.columns
print(patient_id)
print(recording_loc)
print(murmur)
print(murmur_loc)

[2530, 9979, 9983, 13918, 14241, 14998, 23625, 24160, 29045, 29378, 31737, 33151, 36327, 38337, 39043, 39403, 39456, 40058, 40798, 40840, 43852, 44514, 45843, 46065, 46532, 46579, 46778, 47002, 49558, 49561, 49562, 49568, 49572, 49574, 49577, 49585, 49595, 49598, 49607, 49610, 49618, 49622, 49627, 49628, 49630, 49631, 49638, 49641, 49653, 49659, 49661, 49669, 49678, 49683, 49687, 49691, 49704, 49712, 49719, 49729, 49735, 49745, 49748, 49751, 49754, 49761, 49776, 49808, 49821, 49823, 49824, 49829, 49832, 49838, 49839, 49842, 49850, 49853, 49854, 49873, 49876, 49896, 49897, 49900, 49930, 49931, 49946, 49952, 49959, 49960, 49963, 49966, 49968, 49969, 49970, 49974, 49978, 49979, 49980, 49983, 49986, 49987, 49988, 49989, 49990, 49993, 49994, 49995, 49998, 49999, 50001, 50004, 50005, 50006, 50007, 50008, 50009, 50012, 50014, 50015, 50017, 50018, 50023, 50026, 50027, 50029, 50030, 50032, 50034, 50037, 50043, 50047, 50048, 50049, 50053, 50054, 50056, 50057, 50061, 50066, 50067, 50070, 50072, 5

In [4]:
recording_loc = [x.split("+") for x in recording_loc]

for i in range(len(murmur_loc)):
    if murmur_loc[i] is np.nan:
        murmur_loc[i] = []
    else:
        murmur_loc[i] = murmur_loc[i].split("+")

In [5]:
X = []
y = []

for i in range(len(patient_id)):
    if murmur[i] == "Absent":
        for recloc in recording_loc[i]:
            full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_MFCC.csv", delimiter=',')
            initial = 0
            while (initial+400) <= len(full_recording):
                X.append(full_recording[initial:initial+400])
                y.append([1, 0, 0, 0, 0])
                initial += 200
    elif murmur[i] == "Present":
        for recloc in recording_loc[i]:
            if recloc in murmur_loc[i]:
                full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_MFCC.csv", delimiter=',')
                initial = 0
                if recloc == "AV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 1, 0, 0, 0])
                        initial += 200
                elif recloc == "PV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 1, 0, 0])
                        initial += 200
                elif recloc == "TV":
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 1, 0])
                        initial += 200
                else:
                    while (initial+400) <= len(full_recording):
                        X.append(full_recording[initial:initial+400])
                        y.append([0, 0, 0, 0, 1])
                        initial += 200
            else:
                full_recording = np.loadtxt("SSE/" + str(patient_id[i]) + "_" + recloc + "_MFCC.csv", delimiter=',')
                initial = 0
                while (initial+400) <= len(full_recording):
                    X.append(full_recording[initial:initial+400])
                    y.append([1, 0, 0, 0, 0])
                    initial += 200
    print("Completed patient #", str(i+1))

Completed patient # 1
Completed patient # 2
Completed patient # 3
Completed patient # 4
Completed patient # 5
Completed patient # 6
Completed patient # 7
Completed patient # 8
Completed patient # 9
Completed patient # 10
Completed patient # 11
Completed patient # 12
Completed patient # 13
Completed patient # 14
Completed patient # 15
Completed patient # 16
Completed patient # 17
Completed patient # 18
Completed patient # 19
Completed patient # 20
Completed patient # 21
Completed patient # 22
Completed patient # 23
Completed patient # 24
Completed patient # 25
Completed patient # 26
Completed patient # 27
Completed patient # 28
Completed patient # 29
Completed patient # 30
Completed patient # 31
Completed patient # 32
Completed patient # 33
Completed patient # 34
Completed patient # 35
Completed patient # 36
Completed patient # 37
Completed patient # 38
Completed patient # 39
Completed patient # 40
Completed patient # 41
Completed patient # 42
Completed patient # 43
Completed patient # 

Completed patient # 349
Completed patient # 350
Completed patient # 351
Completed patient # 352
Completed patient # 353
Completed patient # 354
Completed patient # 355
Completed patient # 356
Completed patient # 357
Completed patient # 358
Completed patient # 359
Completed patient # 360
Completed patient # 361
Completed patient # 362
Completed patient # 363
Completed patient # 364
Completed patient # 365
Completed patient # 366
Completed patient # 367
Completed patient # 368
Completed patient # 369
Completed patient # 370
Completed patient # 371
Completed patient # 372
Completed patient # 373
Completed patient # 374
Completed patient # 375
Completed patient # 376
Completed patient # 377
Completed patient # 378
Completed patient # 379
Completed patient # 380
Completed patient # 381
Completed patient # 382
Completed patient # 383
Completed patient # 384
Completed patient # 385
Completed patient # 386
Completed patient # 387
Completed patient # 388
Completed patient # 389
Completed patien

Completed patient # 693
Completed patient # 694
Completed patient # 695
Completed patient # 696
Completed patient # 697
Completed patient # 698
Completed patient # 699
Completed patient # 700
Completed patient # 701
Completed patient # 702
Completed patient # 703
Completed patient # 704
Completed patient # 705
Completed patient # 706
Completed patient # 707
Completed patient # 708
Completed patient # 709
Completed patient # 710
Completed patient # 711
Completed patient # 712
Completed patient # 713
Completed patient # 714
Completed patient # 715
Completed patient # 716
Completed patient # 717
Completed patient # 718
Completed patient # 719
Completed patient # 720
Completed patient # 721
Completed patient # 722
Completed patient # 723
Completed patient # 724
Completed patient # 725
Completed patient # 726
Completed patient # 727
Completed patient # 728
Completed patient # 729
Completed patient # 730
Completed patient # 731
Completed patient # 732
Completed patient # 733
Completed patien

In [6]:
time_steps = 400
feature_length = 13
batch_size = 3

In [7]:
from sklearn.model_selection import train_test_split as tts
X_train, X_test, y_train, y_test = tts(X, y, test_size = 0.3)

In [8]:
X_train_final = []
y_train_final = []

for i in range(len(X_train)):
    if y_train[i] == [1, 0, 0, 0, 0]:
        #randnum = random.uniform(0, 1)
        randnum = random.random()
        if randnum >= 0.66:
            X_train_final.append(X_train[i])
            y_train_final.append(y_train[i])
    else:
        X_train_final.append(X_train[i])
        y_train_final.append(y_train[i])

In [9]:
print(len(X_train), len(y_train))
print(len(X_train_final), len(y_train_final))
print(len(X_test), len(y_test))

20864 20864
9374 9374
8942 8942


In [10]:
model = Sequential()
model.add(LSTM(13))
model.add(Dense(13, activation="sigmoid"))
model.add(Dense(128, activation="sigmoid"))
model.add(Dense(16, activation="sigmoid"))
model.add(Dense(5, activation="sigmoid"))

In [11]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [12]:
X_train = np.array(X_train)
y_train = np.array(y_train)
X_train_final = np.array(X_train_final)
y_train_final = np.array(y_train_final)
X_test = np.array(X_test)
y_test = np.array(y_test)

In [13]:
model.fit(X_train_final, y_train_final, epochs=10, batch_size=batch_size)

Epoch 1/10


3125/3125 [==============================] - 203s 64ms/step - loss: 0.3792 - accuracy: 0.6291
Epoch 2/10
3125/3125 [==============================] - 202s 65ms/step - loss: 0.3560 - accuracy: 0.6308
Epoch 3/10
3125/3125 [==============================] - 199s 64ms/step - loss: 0.3476 - accuracy: 0.6351
Epoch 4/10
3125/3125 [==============================] - 191s 61ms/step - loss: 0.3452 - accuracy: 0.6400
Epoch 5/10
3125/3125 [==============================] - 234s 75ms/step - loss: 0.3413 - accuracy: 0.6399
Epoch 6/10
3125/3125 [==============================] - 134s 43ms/step - loss: 0.3394 - accuracy: 0.6380
Epoch 7/10
3125/3125 [==============================] - 150s 48ms/step - loss: 0.3367 - accuracy: 0.6376
Epoch 8/10
3125/3125 [==============================] - 115s 37ms/step - loss: 0.3348 - accuracy: 0.6410
Epoch 9/10
3125/3125 [==============================] - 117s 37ms/step - loss: 0.3313 - accuracy: 0.6457
Epoch 10/10
3125/3125 [==============================

In [14]:
final_final_predicts = model.predict(X_test)
final_final_predicts.shape

280/280 [==============================] - 3s 10ms/step


(8942, 5)

In [15]:
X = np.array(X)
y = np.array(y)
score_full = model.evaluate(X, y, batch_size=batch_size, verbose=0)
print('Test loss:', score_full[0])
print('Accuracy:', score_full[1])

Test loss: 0.21596893668174744
Accuracy: 0.8342615365982056


In [16]:
score = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
print('Test loss:', score[0])
print('Accuracy:', score[1])

Test loss: 0.21752558648586273
Accuracy: 0.8340415954589844


In [17]:
final_final_predicts

array([[0.7019858 , 0.07232907, 0.06670047, 0.0760471 , 0.09552359],
       [0.79958594, 0.04928871, 0.04658784, 0.05468056, 0.06622802],
       [0.72521013, 0.06696532, 0.06176821, 0.0702478 , 0.08818347],
       ...,
       [0.82378703, 0.04086835, 0.03919244, 0.05003255, 0.05808118],
       [0.83489156, 0.03473613, 0.0331988 , 0.04662634, 0.05329775],
       [0.7978294 , 0.04990005, 0.047043  , 0.05481647, 0.06683082]],
      dtype=float32)

In [18]:
for i in range(len(final_final_predicts)):
    if max(final_final_predicts[i]) != final_final_predicts[i][0]:
        print(i, final_final_predicts[i])

12 [0.14345144 0.1556607  0.20649242 0.2283988  0.24729504]
42 [0.08334088 0.1468488  0.2394266  0.2515394  0.26961696]
86 [0.04735489 0.13961552 0.27223846 0.26890394 0.29193148]
98 [0.05336652 0.14097935 0.26555723 0.26581693 0.28735653]
121 [0.07868706 0.14614552 0.2430237  0.25361198 0.27203122]
139 [0.04140193 0.1384506  0.2796139  0.27174145 0.29706427]
150 [0.05322053 0.14088519 0.26569107 0.26594532 0.28744018]
165 [0.07956019 0.14611834 0.24230644 0.25328502 0.27157995]
176 [0.21015032 0.16137876 0.18171649 0.20727004 0.2303568 ]
205 [0.06044403 0.1423715  0.2584689  0.2623916  0.28249586]
230 [0.08665192 0.1481325  0.23739094 0.2497625  0.26830342]
246 [0.07428607 0.1456644  0.24643551 0.25539657 0.27439544]
255 [0.21179837 0.1621841  0.1810771  0.20571719 0.2303455 ]
259 [0.07044899 0.14438623 0.24952051 0.25748178 0.27645862]
268 [0.05454271 0.14103733 0.2642677  0.26538995 0.28644028]
285 [0.09849146 0.14947458 0.22964044 0.24511905 0.2630242 ]
322 [0.14618704 0.15596418 0

In [19]:
full_predicts = model.predict(X)
full_predicts

932/932 [==============================] - 9s 10ms/step


array([[0.7410445 , 0.06270544, 0.05604724, 0.06158458, 0.08286075],
       [0.8214324 , 0.04187391, 0.04011713, 0.0505438 , 0.05895387],
       [0.81231976, 0.04550795, 0.04345394, 0.05260026, 0.06224208],
       ...,
       [0.80124795, 0.0486479 , 0.0463889 , 0.05519559, 0.06582388],
       [0.80384636, 0.04819902, 0.04582139, 0.0542497 , 0.0650209 ],
       [0.61439466, 0.09110261, 0.07513876, 0.07923476, 0.12078968]],
      dtype=float32)

In [20]:
for i in range(len(full_predicts)):
    if max(full_predicts[i]) != full_predicts[i][0]:
        print(i, full_predicts[i])

35 [0.16709366 0.1589472  0.19752695 0.2209766  0.24099958]
38 [0.18608315 0.1600693  0.19019505 0.21522616 0.2358835 ]
39 [0.12653323 0.15429445 0.21498689 0.23457915 0.25294432]
50 [0.08538561 0.14739585 0.2380763  0.25026804 0.26880226]
51 [0.05858416 0.14186633 0.2601749  0.26337543 0.2836478 ]
52 [0.06061796 0.14242586 0.25824112 0.2622604  0.28235006]
53 [0.07941791 0.14626819 0.24233432 0.25293854 0.27168116]
54 [0.05550054 0.14130501 0.2632752  0.26482654 0.2857834 ]
55 [0.05506251 0.141137   0.26371014 0.2651309  0.28606138]
56 [0.08507561 0.14738326 0.23860942 0.2510145  0.26895538]
57 [0.04916056 0.13994305 0.27012804 0.2680309  0.2904743 ]
58 [0.06006541 0.14227408 0.2588398  0.26270893 0.28269926]
59 [0.15555212 0.15711352 0.20173565 0.22473599 0.24374092]
60 [0.07651675 0.14558549 0.2445692  0.25467387 0.273091  ]
61 [0.05897281 0.14215575 0.25985613 0.26305306 0.2834565 ]
62 [0.0496608  0.14022124 0.26960155 0.26766518 0.2901287 ]
63 [0.0475767  0.13968202 0.27197075 0.2